This notebook assumes that you have a running instance of Neo4J Aura. You can create a free one by heading over to https://neo4j.com/

Once you have your instance, save its id as `neo4j_instance_id` and its password as `neo4j_password` as Colab secrets and grant this notebook access to said secrets.

In [1]:
!pip install neo4j yfiles_jupyter_graphs_for_neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.3 MB/s eta 0:00:00


In [2]:
from neo4j import GraphDatabase

In [4]:
from google.colab import userdata
neo4j_instance_id = userdata.get('neo4j_instance_id')
neo4j_password = userdata.get('neo4j_password')

driver = GraphDatabase.driver(f"neo4j+s://{neo4j_instance_id}.databases.neo4j.io:7687", auth=("neo4j", neo4j_password))

In [5]:
def insert_data(tx):
    query = """
    // Create account holders
    CREATE (accountHolder1:AccountHolder {
            FirstName: "John",
            LastName: "Doe",
            UniqueId: "JohnDoe" })

    CREATE (accountHolder2:AccountHolder {
            FirstName: "Jane",
            LastName: "Appleseed",
            UniqueId: "JaneAppleseed" })

    CREATE (accountHolder3:AccountHolder {
            FirstName: "Matt",
            LastName: "Smith",
            UniqueId: "MattSmith" })

    // Create Address
    CREATE (address1:Address {
            Street: "123 NW 1st Street",
            City: "San Francisco",
            State: "California",
            ZipCode: "94101" })

    // Connect 3 account holders to 1 address
    CREATE (accountHolder1)-[:HAS_ADDRESS]->(address1),
           (accountHolder2)-[:HAS_ADDRESS]->(address1),
           (accountHolder3)-[:HAS_ADDRESS]->(address1)

    // Create Phone Number
    CREATE (phoneNumber1:PhoneNumber { PhoneNumber: "555-555-5555" })

    // Connect 2 account holders to 1 phone number
    CREATE (accountHolder1)-[:HAS_PHONENUMBER]->(phoneNumber1),
           (accountHolder2)-[:HAS_PHONENUMBER]->(phoneNumber1)

    // Create SSN
    CREATE (ssn1:SSN { SSN: "241-23-1234" })

    // Connect 2 account holders to 1 SSN
    CREATE (accountHolder2)-[:HAS_SSN]->(ssn1),
           (accountHolder3)-[:HAS_SSN]->(ssn1)

    // Create SSN and connect 1 account holder
    CREATE (ssn2:SSN { SSN: "241-23-4567" })<-[:HAS_SSN]-(accountHolder1)

    // Create Credit Card and connect 1 account holder
    CREATE (creditCard1:CreditCard {
            AccountNumber: "1234567890123456",
            Limit: 5000, Balance: 1442.23,
            ExpirationDate: "01-20",
            SecurityCode: "123" })<-[:HAS_CREDITCARD]-(accountHolder1)

    // Create Bank Account and connect 1 account holder
    CREATE (bankAccount1:BankAccount {
            AccountNumber: "2345678901234567",
            Balance: 7054.43 })<-[:HAS_BANKACCOUNT]-(accountHolder1)

    // Create Credit Card and connect 1 account holder
    CREATE (creditCard2:CreditCard {
            AccountNumber: "1234567890123456",
            Limit: 4000, Balance: 2345.56,
            ExpirationDate: "02-20",
            SecurityCode: "456" })<-[:HAS_CREDITCARD]-(accountHolder2)

    // Create Bank Account and connect 1 account holder
    CREATE (bankAccount2:BankAccount {
            AccountNumber: "3456789012345678",
            Balance: 4231.12 })<-[:HAS_BANKACCOUNT]-(accountHolder2)

    // Create Unsecured Loan and connect 1 account holder
    CREATE (unsecuredLoan2:UnsecuredLoan {
            AccountNumber: "4567890123456789-0",
            Balance: 9045.53,
            APR: .0541,
            LoanAmount: 12000.00 })<-[:HAS_UNSECUREDLOAN]-(accountHolder2)

    // Create Bank Account and connect 1 account holder
    CREATE (bankAccount3:BankAccount {
            AccountNumber: "4567890123456789",
            Balance: 12345.45 })<-[:HAS_BANKACCOUNT]-(accountHolder3)

    // Create Unsecured Loan and connect 1 account holder
    CREATE (unsecuredLoan3:UnsecuredLoan {
            AccountNumber: "5678901234567890-0",
            Balance: 16341.95, APR: .0341,
            LoanAmount: 22000.00 })<-[:HAS_UNSECUREDLOAN]-(accountHolder3)

    // Create Phone Number and connect 1 account holder
    CREATE (phoneNumber2:PhoneNumber {
            PhoneNumber: "555-555-1234" })<-[:HAS_PHONENUMBER]-(accountHolder3)

    RETURN *
    """
    tx.run(query)

# Run the query
with driver.session() as session:
    session.execute_write(insert_data)

In [8]:
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget

g = Neo4jGraphWidget(driver)

def show_graph(driver):
    query = """
    MATCH (n)-[r]->(m)
    RETURN n, r, m
    """
    g.show_cypher(query)

show_graph(driver)


GraphWidget(layout=Layout(height='690px', width='100%'))

In [9]:
def show_fraud_ring(driver):
    query = """
      MATCH (accountHolder:AccountHolder)-[r]->(contactInformation)
      WITH contactInformation, count(accountHolder) AS RingSize, collect(r) AS relationships
      WHERE RingSize > 1
      MATCH (contactInformation)<-[r2]-(accountHolder)
      RETURN accountHolder, contactInformation, r2, relationships
    """
    g.show_cypher(query)

show_fraud_ring(driver)

GraphWidget(layout=Layout(height='500px', width='100%'))

In [10]:
import pandas as pd

def show_financial_risk(driver):
    query = """
      MATCH 		(accountHolder:AccountHolder)-[]->(contactInformation)
      WITH 		contactInformation,
            count(accountHolder) AS RingSize
      MATCH 		(contactInformation)<-[]-(accountHolder),
            (accountHolder)-[r:HAS_CREDITCARD|HAS_UNSECUREDLOAN]->(unsecuredAccount)
      WITH 		collect(DISTINCT accountHolder.UniqueId) AS AccountHolders,
            contactInformation, RingSize,
            SUM(CASE type(r)
              WHEN 'HAS_CREDITCARD' THEN unsecuredAccount.Limit
              WHEN 'HAS_UNSECUREDLOAN' THEN unsecuredAccount.Balance
              ELSE 0
            END) as FinancialRisk
      WHERE 		RingSize > 1
      RETURN 		AccountHolders AS FraudRing,
            labels(contactInformation) AS ContactType,
            RingSize,
            round(FinancialRisk) as FinancialRisk
      ORDER BY 	FinancialRisk DESC
    """
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

results = show_financial_risk(driver)
pd.DataFrame(results)

,FraudRing,ContactType,RingSize,FinancialRisk
0,"[JohnDoe, JaneAppleseed, MattSmith]",[Address],3,34387.0
1,"[JaneAppleseed, MattSmith]",[SSN],2,29387.0
2,"[JohnDoe, JaneAppleseed]",[PhoneNumber],2,18046.0


In [7]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [11]:
def clear_graph(tx):
    query = """
    MATCH (n)
    DETACH DELETE n
    """
    tx.run(query)

with driver.session() as session:
    session.execute_write(clear_graph)